#Importing Libraries

In [ ]:
import torch                                                              #Core PyTorch Package
import torch.nn as nn                                                     #Contains modules to build neural networks
import torch.optim as optim                                               #has optimization algorithms
import torchvision                                                        #Utilities for vision tasks like loading datasets, pretrained models
import torchvision.transforms as transforms                               #Tools for preprocessing like resizing, normalization
from torch.utils.data import DataLoader                                   #Batches, shuffles, and loads the data in parallel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")     #Set GPU for computation
print("Using device:",device)

Using device: cuda


#Data Loading and Preprocessing

In [ ]:
transform = transforms.Compose([                                           #Chains multiple transformations
    transforms.ToTensor(),                                                 #Converts images from [0,255] PIL format of [0,1] tensors and rearranges the dimension from (H,W,C) to (C,H,W)
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))                      #Normalizes each channel using (x - mean)/std | shifts pixel range from [0,1] → [-1,1]
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:04<00:00, 37.0MB/s]


#Defining CNN architecture

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels = 6, kernel_size=5)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride = 2)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size= 5)
    self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)

    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)
    self.relu = nn.ReLU()
    self.flatten = nn.Flatten()
    # self.softmax = nn.Softmax()                            ##We don't add softmax activation because the cross entropy loss function expects direct logits.
                                                             ## It adds softmax and log function to it

  def forward(self,x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.relu(x)
    x = self.pool2(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)

    return x

cnn = CNN().to(device)

##Choosing Loss Function and Optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr = 0.001, momentum = 0.9)

# W&B integration

In [ ]:
!pip install wandb

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: samdanikshitij21 (samdanikshitij21-indian-institute-of-technology-delhi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import wandb

In [ ]:
import yaml

with open("config.yaml") as f:
    config = yaml.safe_load(f)

wandb.init(project="basic_cnn", config=config)

In [ ]:
# wandb.init(
#     project="basic_cnn",     # name of the project
#     config={
#         "epochs": 30,
#         "batch_size": 64,
#         "learning_rate": 0.001,
#         "momentum": 0.9,
#         "optimizer": "SGD"
#         "criterion": "cross_entropy"
#         "architecture": "CNN"
#     }
# )

#Training

In [ ]:
def train(model, dataloader, criterion, optimize,epochs):
  model.train()
  for epoch in range(epochs):
    total_loss = 0
    correct_pred = 0
    total_pred = 0

    for images,labels in dataloader:
      images,labels = images.to(device), labels.to(device)

      outputs = model(images)         ##also called logits

      optimize.zero_grad()
      loss = criterion(outputs,labels)
      loss.backward()
      optimize.step()

      total_loss += loss.item()*images.size(0)

      _,predicted = torch.max(outputs, dim = 1)
      correct_pred += (predicted==labels).sum().item()
      total_pred += labels.size(0)
      accuracy = correct_pred/total_pred
      wandb.log({"batch_loss": loss.item()*images.size(0), "accuracy":accuracy, "epoch": epoch})

    total_epoch_loss = total_loss/total_pred
    epoch_accuracy = correct_pred/total_pred
    wandb.log({"train_loss": total_epoch_loss, "epoch": epoch})
    print(f"Epoch {epoch + 1}, Loss: {total_epoch_loss}, Accuracy: {epoch_accuracy:.4f}")

In [ ]:
train(cnn,trainloader,loss_fn,optimizer,wandb.config.epochs)

Epoch 1, Loss: 2.296700517425537, Accuracy: 0.1278
Epoch 2, Loss: 2.172718389968872, Accuracy: 0.2237
Epoch 3, Loss: 1.9858435398101806, Accuracy: 0.2974
Epoch 4, Loss: 1.7925050327301026, Accuracy: 0.3515
Epoch 5, Loss: 1.6373164974594117, Accuracy: 0.4042
Epoch 6, Loss: 1.5355887184524537, Accuracy: 0.4423
Epoch 7, Loss: 1.4703206390380859, Accuracy: 0.4657
Epoch 8, Loss: 1.417445563659668, Accuracy: 0.4883
Epoch 9, Loss: 1.365724368019104, Accuracy: 0.5088
Epoch 10, Loss: 1.3205928610229491, Accuracy: 0.5279
Epoch 11, Loss: 1.2795629123306274, Accuracy: 0.5440
Epoch 12, Loss: 1.2397749897766113, Accuracy: 0.5587
Epoch 13, Loss: 1.2121686199188233, Accuracy: 0.5693
Epoch 14, Loss: 1.180049778442383, Accuracy: 0.5797
Epoch 15, Loss: 1.148559942970276, Accuracy: 0.5917
Epoch 16, Loss: 1.121880997390747, Accuracy: 0.6017
Epoch 17, Loss: 1.099302931137085, Accuracy: 0.6106
Epoch 18, Loss: 1.0746346551513672, Accuracy: 0.6201
Epoch 19, Loss: 1.055187281780243, Accuracy: 0.6257
Epoch 20, L

In [ ]:
def evaluate(model,dataloader,criterion):
  model.eval()
  test_loss = 0
  correct_pred = 0
  total_pred = 0

  with torch.no_grad():
    for images,labels in dataloader:
      images,labels = images.to(device), labels.to(device)   ##labels.shape = [B]

      outputs = model(images)

      loss = criterion(outputs,labels)

      test_loss += loss.item()*images.size(0)     ##loss.item() gives the scalar value of loss tensor

      _,predicted = torch.max(outputs, dim = 1)   ##outputs.shape = [B,10]
      correct_pred += (predicted==labels).sum().item()  ##predicted.shape = [B]
      total_pred += labels.size(0)

  test_loss /= total_pred
  test_acc = correct_pred/total_pred
  wandb.log({"val_loss": test_loss, "val_acc": test_acc})
  return test_loss, test_acc

In [ ]:
val_loss,val_accuracy = evaluate(cnn,testloader,loss_fn)
print('Accuracy:', val_accuracy)

Accuracy: 0.6311
